# Load packages

In [40]:
%load_ext autoreload
%autoreload 2

import importlib

# custom packages
from src.gym2048 import gym2048
from src.models import CNN22
from src.RL import TDL

importlib.reload(importlib.import_module('src.gym2048'))
importlib.reload(importlib.import_module('src.RL'))

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle
import cProfile
import pstats

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load and test environment

In [1]:
env = gym2048()

NameError: name 'gym2048' is not defined

In [ ]:
env.check()


 Check SwipeLeft 

[ True  True  True  True] True
[ True  True  True  True] True
[ True  True  True  True] True
[ True  True  True  True] True
[ True  True  True  True] True

 Check SwipeLeft 

[[0 0 2 2]
 [0 2 2 2]
 [2 0 2 0]
 [0 2 0 0]]
[[4 0 0 0]
 [4 2 0 0]
 [4 0 0 0]
 [2 0 0 0]]
[[0 0 0 4]
 [0 0 2 4]
 [0 0 0 4]
 [0 0 0 2]]
[[2 4 4 4]
 [0 0 2 0]
 [0 0 0 0]
 [0 0 0 0]]
[[0 0 0 0]
 [0 0 0 0]
 [0 0 2 0]
 [2 4 4 4]]
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]] True
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]] True
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]] True
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]] True
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]

 Check encode 

# Training

In [ ]:
#env = Monitor(env, filename="./logs/tdl_cnn_2024_11_14_1.csv")
model = TDL(CNN22, env, device="mps")
#model.load_network('./TDL_CNN_2024_11_10_1/rl_model_10000_epsiodes.zip')

In [ ]:
#profiler = cProfile.Profile()
#profiler.enable()

model.learn(
    1e7, 
    learning_rate=1e-3, 
    gamma=0.99, 
    buffer_size=1024, 
    exploration_initial_eps=0.0, 
    exploration_final_eps=0.0, 
    exploration_fraction=0.1, 
    train_freq=1024, 
    batch_size=1024, 
    target_network_update_freq=-1, # -1 for no target network 
    prioritized_replay=False, 
    prioritized_replay_alpha=0.6, 
    prioritized_replay_beta0=0.4, 
    prioritized_replay_beta_increment=1e-4, 
    prioritized_replay_eps=1e-6,
    prioritized_replay_update_freq=500, 
    checkpoint_path='./exp/TDL_CNN_2024_11_18/', 
    checkpoint_freq=2500, 
    check_gradient_freq=500, 
    restart_jump=False, 
    use_symmetry=True
    )

#profiler.disable()
#stats = pstats.Stats(profiler)

# Evaluation

In [ ]:
profiler = cProfile.Profile()
profiler.enable()

env = gym2048()
model = TDL(CNN22, env)
model.load_network('./exp/TDL_CNN_2024_11_18/rl_model_10000_epsiodes.zip')

model.eval()

model.env.playOneGame(policy=model.select_expectimax_action, verbose=True, depth=2)

profiler.disable()
stats = pstats.Stats(profiler)
stats.dump_stats('./profile_results.prof')

Exception ignored When destroying _lsprof profiler:
Traceback (most recent call last):
  File "/var/folders/_6/22xfh5m12w79yz7j4yt010fw0000gn/T/ipykernel_86241/1243807404.py", line 2, in <module>
RuntimeError: Cannot install a profile function while another profile function is being installed


In [ ]:
def get_param_stats(model):
    stats = {
        "Layer": [],
        "Min": [],
        "Max": [],
        "Mean": [],
        "Std": [],
        "Num Params": []
    }

    for name, param in model.named_parameters():
        if param.requires_grad:
            stats["Layer"].append(name)
            stats["Min"].append(param.data.min().item())
            stats["Max"].append(param.data.max().item())
            stats["Mean"].append(param.data.mean().item())
            stats["Std"].append(param.data.std().item())
            stats["Num Params"].append(param.numel())

    # Convert to DataFrame for easy viewing
    stats_df = pd.DataFrame(stats)
    return stats_df

# Example usage
model_stats = get_param_stats(model.policy_network)
print(model_stats)

In [ ]:

model_max_tiles, model_n_steps = env.evaluate(model=model, n_games=1000)

data = {
    'max_tiles': model_max_tiles,
    'n_steps': model_n_steps
    }

with open('./exp/TDL_CNN_2024_11_18/stats_10000_episodes.pkl', 'wb') as file:
    pickle.dump(data, file)

# Comparison

In [ ]:
models = {
    'Random': './exp/2048_random.pkl',
    'TDL_nov14': './exp/TDL_CNN_2024_11_14_1/stats_10000_episodes.pkl',
    'TDL_nov16': './exp/TDL_CNN_2024_11_16/stats_10000_episodes.pkl',
    'TDL_nov18': './exp/TDL_CNN_2024_11_18/stats_10000_episodes.pkl'
}

df = pd.DataFrame()

for id, path in models.items():
    with open(path, 'rb') as file:
        data = pickle.load(file)
    df_loc = pd.DataFrame({'max_tiles':data['max_tiles']})
    df_loc['model'] = id
    df = pd.concat([df, df_loc], ignore_index=True)

df = (df.groupby(['max_tiles', 'model']).size()/10).reset_index(name='counts')

plt.figure(figsize=(12, 6))
sns.barplot(data=df, x='max_tiles', y='counts', hue='model')
plt.xlabel('Max tile')
plt.ylabel('Percent')
plt.tight_layout()
plt.grid(linestyle='--')
plt.show()

In [ ]:
env = gym2048()
env.reset()
print(env.state[0])
print(env.decode(env.state))
zero_idx = np.argwhere(env.state[0] == 1)
zero_idx
for idx in zero_idx:
    print(idx[0], idx[1])
print(zero_idx.shape[0])

In [ ]:
all([True, False])